In [1]:
import json 
import redis
import cherrypy

In [2]:
#open redis connection

REDIS_HOST = "redis-17637.c114.us-east-1-4.ec2.cloud.redislabs.com"
REDIS_PORT = 17637
REDIS_USER = "default"
REDIS_PASSWORD = "fFh38GR6BxHI0xVoGbsUKAzPZlCBXLAA"

redis_client = redis.Redis(host = REDIS_HOST, port = REDIS_PORT, password = REDIS_PASSWORD, username= REDIS_USER)
print("Is connected:", redis_client.ping())


Is connected: True


In [3]:
#endpoint /devices
class Devices:
    exposed = True

    def GET(self, *path, **query):
        keys = redis_client.keys("*:ambulance")
        keys = [k.decode().split(':')[0] for k in keys]
        output_dict = {"mac_addresses" : keys}
        return json.dumps(output_dict)



#endpoint /devices/{mac_address} 
class DeviceList:
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad request: missing MAC address.')
        
        if len(redis_client.keys(f"{path[0]}:*"))==0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')

        if query.get("from") is None:
            raise cherrypy.HTTPError(400, 'Bad request: missing start time.')
        if query.get("to") is None:
            raise cherrypy.HTTPError(400, 'Bad request: missing end time.')

    
        ambulances = redis_client.ts().range(f"{path[0]}:ambulance", query.get("from"), query.get("to"))

        output_dict = {
            "mac_address" : path[0],
            "timestamps" : [ts for ts, bl in ambulances],
            "ambulance" : [bl for ts, bl in ambulances]
        }

        return json.dumps(output_dict)

    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad request: missing MAC address.')

        if len(redis_client.keys(f"{path[0]}:*"))==0:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address.')
        
        redis_client.delete(f"{path[0]}:ambulance")


In [4]:
if __name__== '__main__':
    conf = {
        '/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}
    }

    #create link between the class and the endpoint
    cherrypy.tree.mount(Devices(), '/devices', conf)
    cherrypy.tree.mount(DeviceList(), '/device', conf)
  
    #specify the IP address of the host and host
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080}) 

    #start and keep it online
    cherrypy.engine.start()
    cherrypy.engine.block()

[13/Mar/2023:15:10:42] ENGINE Bus STARTING
[13/Mar/2023:15:10:42] ENGINE Started monitor thread 'Autoreloader'.
[13/Mar/2023:15:10:42] ENGINE Serving on http://0.0.0.0:8080
[13/Mar/2023:15:10:42] ENGINE Bus STARTED
172.3.22.214 - - [13/Mar/2023:15:10:46] "GET /devices HTTP/1.1" 200 55 "" "python-requests/2.28.1"
172.3.22.214 - - [13/Mar/2023:15:10:49] "GET /device/0xc8d200151708?from=1678719864669&to=1678719900716 HTTP/1.1" 200 244 "" "python-requests/2.28.1"
172.3.18.60 - - [13/Mar/2023:15:10:49] "GET /device/0xec1c6e72a03d?from=1678719864669&to=1678719900716 HTTP/1.1" 200 304 "" "python-requests/2.28.1"
172.3.18.60 - - [13/Mar/2023:15:12:04] "GET /devices HTTP/1.1" 200 55 "" "python-requests/2.28.1"
172.3.22.214 - - [13/Mar/2023:15:12:06] "DELETE /device/0xec1c6e72a03d HTTP/1.1" 200 - "" "python-requests/2.28.1"
172.3.22.214 - - [13/Mar/2023:15:12:09] "GET /devices HTTP/1.1" 200 37 "" "python-requests/2.28.1"
[13/Mar/2023:15:24:19] ENGINE Keyboard Interrupt: shutting down bus
[13/Mar

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=957cfe07-d816-4770-9cc0-d7c1b4767886' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>